# simple recommender ( IMDb Top 250 formula)

In [3]:
#current directory 

import os
os.getcwd()


'C:\\Users\\LD.MonoTsango'

In [4]:
#'change directory'
os.chdir('C:\\Users\\LD.MonoTsango\\.spyder-py3\\data')

In [ ]:
# list elements of the current directory
os.listdir()

In [5]:
# import pandas as pd

import pandas as pd

# Load Movies Metadata

movies = pd.read_csv('tmdb_5000_movies.csv', low_memory=False)
credit = pd.read_csv('tmdb_5000_credits.csv', low_memory=False)


In [17]:
# Print the first three rows : movies 
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [38]:
# Print the first three rows : credits
credit.head(3)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [37]:
# Simple Recommender : IMDb Top 250    

# As a first step, let's calculate the value of C, the mean rating across all movies:

# Calculate C

C =  movies['vote_average'].mean()
print(C)




6.092171559442011


In [39]:
#The average rating of a movie on IMDB is around 6.09, on a scale of 10.
#Next, let's calculate the number of votes, m, received by a movie in the 90th percentile. 
#The pandas library makes this task extremely trivial using the .quantile() method of a pandas Series:


# Calculate the minimum number of votes required to be in the chart, m
m = movies['vote_count'].quantile(0.90)
print(m)

#the minimum number of votes required to be in the in the chart is equal to 1838.

1838.4000000000015


In [40]:
# Filter out all qualified movies into a new DataFrame  q_movies
q_movies = movies.copy().loc[movies['vote_count'] >= m]
q_movies.shape


#There are 481 movies qualified, i.e that have a vote count > 1838  ( 90% percentile)

(481, 20)

In [41]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [42]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

In [43]:
#Print the top 10 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)


,title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.059258
662,Fight Club,9413,8.3,7.939256
65,The Dark Knight,12002,8.2,7.920020
3232,Pulp Fiction,8428,8.3,7.904645
96,Inception,13752,8.1,7.863239
3337,The Godfather,5893,8.4,7.851236
95,Interstellar,10867,8.1,7.809479
809,Forrest Gump,7927,8.2,7.803188
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,The Empire Strikes Back,5879,8.2,7.697884


In [ ]:
'''
We have to say that this recommender reflect more the popularity audience, 
since it does not go deep with a better understanding of our attributes such as  overview, 
keywords, genres , cast or crew. 
'''

# Plot description based recommender (movie overview)

In [44]:

#In this section, we  build a system that recommends movies that are similar 
#to a particular movie. More specifically, we will compute pairwise similarity 
#scores for all movies based on their plot descriptions ( overview) and 
#recommend movies based on that similarity score.


#Print plot overviews of the first 5 movies.
movies['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [45]:
'''
In its current form, it is not possible to compute the similarity between any two overviews. To do this, you need to compute the word vectors of each overview or document, as it will be called from now on.

we compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document. This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document) and each column represents a movie, as before.

In its essence, the TF-IDF score is the frequency of a word occurring in a document, down-weighted by the number of documents in which it occurs. 
This is done to reduce the importance of words that occur frequently in plot overviews and therefore, their significance in computing the final similarity score.

scikit-learn gives us a built-in TfIdfVectorizer class that produces the TF-IDF matrix .
'''

#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')


#Replace NaN with an empty string
movies['overview'] = movies['overview'].fillna('')


In [46]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(movies['overview'])


#Output the shape of tfidf_matrix
tfidf_matrix.shape


#tfidf_matrix.todense()

# we see that over 20978 different words were used to describe the 4803 movies in our dataset.


(4803, 20978)

In [49]:
#Since we have used the TF-IDF vectorizer, 
#calculating the dot product will directly give you the cosine similarity score. 
#we use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.


# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [47]:
# recommender model creation  

'''
we are going to define a function that takes in a movie title as an input and outputs 
a list of the 10 most similar movies. Firstly, we need a reverse mapping of movie titles 
and DataFrame indices. In other words, we need a mechanism to identify the index of a movie in the
movie DataFrame, given its title.
'''

#Construct a reverse map of indices and movie titles
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
indices

title
Avatar                                            0
Pirates of the Caribbean: At World's End          1
Spectre                                           2
The Dark Knight Rises                             3
John Carter                                       4
Spider-Man 3                                      5
Tangled                                           6
Avengers: Age of Ultron                           7
Harry Potter and the Half-Blood Prince            8
Batman v Superman: Dawn of Justice                9
Superman Returns                                 10
Quantum of Solace                                11
Pirates of the Caribbean: Dead Man's Chest       12
The Lone Ranger                                  13
Man of Steel                                     14
The Chronicles of Narnia: Prince Caspian         15
The Avengers                                     16
Pirates of the Caribbean: On Stranger Tides      17
Men in Black 3                                   18
The Ho

In [50]:
# Function that takes in movie title as input and outputs most similar movies
# The steps are explained in Team38.pdf3

def get_recommendations(title, cosine_sim=cosine_sim):
    
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]


    # Return the top 10 most similar movies
    return movies['title'].iloc[movie_indices]

In [51]:
get_recommendations('Pulp Fiction') 

3526            The Sting
3194       All or Nothing
3466        Sliding Doors
4624            Locker 13
2917          The Fighter
4036            Antibirth
3491         The Wackness
2849             Nebraska
3504                11:14
3346    Jumping the Broom
Name: title, dtype: object

In [30]:
get_recommendations('Titanic') 

1269                                  Raise the Titanic
2143                                         Ghost Ship
2287                         I Can Do Bad All By Myself
770                                       Event Horizon
4287                                            Niagara
3212                                           The Rose
2902                                           Triangle
4228                        The Ballad of Jack and Rose
171     Master and Commander: The Far Side of the World
104                                            Poseidon
Name: title, dtype: object

In [52]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [ ]:
#we observe that the system has done a decent job of finding movies
#with similar plot descriptions, however the quality of the recommendations
#can be ameliorate.

#For example ,"The Dark Knight Rises" returns all Batman movies while it's more
#likely that the people who liked that movie are more inclined to enjoy other Christopher Nolan
#movies. This is something that cannot be captured by the present recommender. Thats why we are going
#to investigate  other metadata such as keywords, genres and credits  to build an improved recommender system.

# genres, keywords, credits (cast & crew) based recommender

In [ ]:
'''
It goes without saying that the quality of your recommender would be increased with the usage of better metadata. 
That is exactly what you are going to do in this section. 
You are going to build a recommender based on the following metadata: 
the 3 top actors, the director, related genres and the movie plot keywords. 
'''

In [10]:
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
movie_id    4803 non-null int64
title       4803 non-null object
cast        4803 non-null object
crew        4803 non-null object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [11]:
# Convert IDs to int. Required for merging

credit.rename(columns={'movie_id': 'id'}, inplace=True)

credit['id'] = credit['id'].astype('int')
movies['id'] = movies['id'].astype('int')

In [13]:
# Merge credits into your main movie dataframe

movies_full = movies.merge(credit, on=['id' , 'title'])

movies_full.info()
movies_full.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int32
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [16]:
#From the new features, cast, crew and keywords, we need to extract the three most 
#important actors, the director and the keywords associated with that movie.
#Right now, our data is present in the form of "stringified" lists. 
#we need to convert them into a form that is usable .

# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    movies_full[feature] = movies_full[feature].apply(literal_eval)



In [19]:
#Next, we write functions that will help us to extract the required information 
#from each feature. First, we'll import the NumPy package to get access to its NaN constant.
#Next, we can use it to write the get_director() function:


# Import Numpy 
import numpy as np

# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


In [20]:
# Returns the list top 3 elements or entire list; whichever is more.
    
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [21]:
# Define new director, cast, genres and keywords features that are in a suitable form.
movies_full['director'] = movies_full['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    movies_full[feature] = movies_full[feature].apply(get_list)

In [27]:
# Print the new features of the first 3 films

movies_full[['title', 'cast', 'director', 'keywords', 'genres']].head(3)



,title,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


In [28]:
'''
The next step would be to convert the names and keyword instances into lowercase and strip all the spaces between them. 
This is done so that your vectorizer doesn't count the Johnny of "Johnny Depp" and "Johnny Galecki" as the same.
After this processing step, the aforementioned actors will be represented as "johnnydepp" and "johnnygalecki" and 
will be distinct to your vectorizer.
'''


# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [29]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    movies_full[feature] = movies_full[feature].apply(clean_data)
    

In [30]:
#We are now in a position to create your "metadata soup", which is a string that 
#contains all the metadata that you want to feed to your vectorizer (namely actors, director and keywords).    

movies_full.head(10)

def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])



In [31]:
# Create a new soup feature
movies_full['soup'] = movies_full.apply(create_soup, axis=1)    

movies_full[['title','director', 'soup']].head(3)

,title,director,soup
0,Avatar,jamescameron,cultureclash future spacewar samworthington zo...
1,Pirates of the Caribbean: At World's End,goreverbinski,ocean drugabuse exoticisland johnnydepp orland...
2,Spectre,sammendes,spy basedonnovel secretagent danielcraig chris...


In [33]:
'''
The next steps are the same as what we did with our plot description based recommender. 
One important difference is that you use the CountVectorizer() instead of TF-IDF. 
This is because you do not want to down-weight the presence of an actor/director 
if he or she has acted or directed in relatively more movies. 
It doesn't make much intuitive sense.
'''


# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(movies_full['soup'])


# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

array([[1. , 0.3, 0.2, ..., 0. , 0. , 0. ],
       [0.3, 1. , 0.2, ..., 0. , 0. , 0. ],
       [0.2, 0.2, 1. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 1. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 1. ]])

In [34]:
# Reset index of your main DataFrame and construct reverse mapping as before
movies_full = movies_full.reset_index() 
indices = pd.Series(movies_full.index, index=movies_full['title'])


movies_full[['title','genres']].head(5)

,title,genres
0,Avatar,"[action, adventure, fantasy]"
1,Pirates of the Caribbean: At World's End,"[adventure, fantasy, action]"
2,Spectre,"[action, adventure, crime]"
3,The Dark Knight Rises,"[action, crime, drama]"
4,John Carter,"[action, adventure, sciencefiction]"


In [55]:
get_recommendations('Titanic', cosine_sim2)

1081             Revolutionary Road
4247          Me You and Five Bucks
49                 The Great Gatsby
872              All the King's Men
1311                     Angel Eyes
1492                     The Reader
2449          Sense and Sensibility
2661                 Romeo + Juliet
2701                Little Children
2946    What's Eating Gilbert Grape
Name: title, dtype: object

In [54]:
#We can now reuse your get_recommendations() function by passing in the new cosine_sim2 matrix as your second argument.

get_recommendations('The Dark Knight Rises', cosine_sim2)


65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

In [ ]:
'''
we observe that our recommender has been successful in capturing more information due to more metadata and has given us (arguably) better recommendations.
for example, the recommender for the Dark knight rises doesn't output all the Batman movies anymore. 
The recommendations seem to have recognized other Christopher Nolan movies (due to the high weightage given to director) and put them as top recommendations. 


One thing that we notice about our recommendation system is that it recommends movies regardless of ratings and popularity.
It is true that Catwoman has a lot of similar characters as compared to The Batman begins but it was a terrible movie that shouldn't be recommended to anyone.
Many critics consider it to be one of the worst films of all time.(https://en.wikipedia.org/wiki/Catwoman_(film)#cite_note-4)

Therefore, we will add a mechanism to remove bad movies and return movies which are popular and have had a good critical response.

I will take the top 25 movies based on similarity scores and calculate the vote of the 60th percentile movie. 
Then, using this as the value of $m$, we will calculate the weighted rating of each movie using IMDB's formula like we did in the Simple Recommender section.


In [56]:
movies_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 25 columns):
index                   4803 non-null int64
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int32
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null fl

In [65]:
pd.options.mode.chained_assignment = None

def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim2[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movie = movies_full.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date']]
    vote_counts = movie[movie['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movie[movie['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movie[(movie['vote_count'] >= m) & (movie['vote_count'].notnull()) & (movie['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [67]:
improved_recommendations('Titanic')

,title,vote_count,vote_average,release_date,wr
439,Shutter Island,6336,7,2010-02-18,6.795832
351,The Departed,4339,7,2006-10-05,6.729829
49,The Great Gatsby,3769,7,2013-05-10,6.702366
2152,Eternal Sunshine of the Spotless Mind,3652,7,2004-03-19,6.696024
818,Captain Phillips,2454,7,2013-10-10,6.611184
297,Blood Diamond,2281,7,2006-12-07,6.594856
2946,What's Eating Gilbert Grape,919,7,1993-12-17,6.394737
961,The Beach,1233,6,2000-02-11,6.055170
720,Contagion,1325,6,2011-09-08,6.053565
2661,Romeo + Juliet,1374,6,1996-10-31,6.052748


In [66]:
improved_recommendations('The Dark Knight Rises')

,title,vote_count,vote_average,release_date,wr
65,The Dark Knight,12002,8,2008-07-16,7.746586
1196,The Prestige,4391,8,2006-10-19,7.436968
119,Batman Begins,7359,7,2005-06-10,6.818541
510,Children of Men,2071,7,2006-09-22,6.573093
280,Public Enemies,1344,6,2009-07-01,6.053245
405,The Fast and the Furious: Tokyo Drift,1705,6,2006-06-03,6.047821
747,Gangster Squad,1778,6,2013-01-10,6.046855
299,Batman Forever,1498,5,1995-05-31,5.601801
157,Exodus: Gods and Kings,1921,5,2014-12-03,5.534087
72,Suicide Squad,7458,5,2016-08-02,5.215981


In [ ]:
'''
Now we can see that the recommendations seem to have recognized not only other Christopher Nolan movies , not only fill with 
the 3 top actors, the director, related genres and the movie plot keywords characteristics,  but also the popularity of the movie.
'''